[View in Colaboratory](https://colab.research.google.com/github/opooladz/ESPRIT-Autoencoder-Anomaly-Detection/blob/master/Copy_of_ESPRIT2.ipynb)

In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  accountsservice acpid activity-log-manager adwaita-icon-theme apg aspell
  aspell-en at-spi2-core avahi-daemon avahi-utils bbswitch-dkms bind9-host
  binfmt-support bluez bluez-obexd bsdmainutils ca-certificates-java
  cheese-common clang-3.8 cracklib-runtime crda cups-pk-helper dbus
  dbus-user-session dbus-x11 dconf-cli dconf-gsettings-backend dconf-service
  default-jre default-jre-headless desktop-file-utils dictionaries-common
  distro-info-data dkms dmsetup dns-root-data dnsmasq-base dosfstools eject
  emacsen-common enchant evolution-data-server evolution-data-server-common
  fakeroot file fontconfig fonts-dejavu-extra fuse gcr gdisk geoclue
  geoclue-ubuntu-geoip geoip-database gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-glib-2.0 gir1.2-gtk-3.0
  gir1.2-ibus-1.0 gir1.2-notify-0.7 gir1.2-packagekit

In [0]:
!pip install numba

    100% |████████████████████████████████| 1.9MB 7.9MB/s 
    100% |████████████████████████████████| 15.9MB 2.5MB/s 


In [1]:
import numpy as np
from scipy.linalg import toeplitz
from numpy import linalg as lg
from time import time
from typing import Tuple
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from numba import jit

flatten = lambda l: [item for sublist in l for item in sublist]

@jit
def compute_autocovariance(x: np.ndarray, M: int) -> np.ndarray:
    """
    This function compute the auto-covariance matrix of a numpy signal.
    The auto-covariance is computed as follows
    .. math:: \textbf{R}=\frac{1}{N}\sum_{M-1}^{N-1}\textbf{x}_{m}\textbf{x}_{m}^{H}
    where :math:`\textbf{x}_{m}^{T}=[x[m],x[m-1],x[m-M+1]]`.
    :param x: 1-D vector of size N
    :param M:  int, optional. Size of signal block.
    :returns: NxN ndarray
    """

    # Create covariance matrix for psd estimation
    # length of the vector x
    x = np.asarray(x).squeeze()
    assert x.ndim == 1, '1-D only'
    N = x.size

    # Create column vector (Nx1) from row array
    x_vect = x[None, :].T

    # init covariance matrix
    yn = x_vect[M-1::-1]  # reverse order from M-1 to 0

    R = yn @ yn.conj().T  # zeroth lag
    # about 5-8% of computation time
    for i in range(1, N-M):  # no zero because we just computed it
        # extract the column vector
        yn = x_vect[M-1+i:i-1:-1]

        R = R + yn @ yn.conj().T

    return R / N
@jit
def wrapper(x):
  return np.convolve(x,xc,mode="valid")

@jit
def esprit1(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time        
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    #U, S, V = lg.svd(R)  # 25% of computation time
    w, v = lg.eig(R)    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = v[:-1, :L]
    # Remove first row
    S2 = v[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)
    t = np.arange(0, 0.01, 1/fs)
    
    x2 = np.exp(-1j*2*np.pi*t[::-1])[:,np.newaxis]**f
    #display(xc.shape)
    #ampCisoid = np.apply_along_axis(lambda q: np.convolve(q,xc,mode="valid"),axis = 0,arr=x2)
    ampCisoid = np.apply_along_axis(wrapper,axis = 0,arr=x2)
    return f, np.abs(w[:L]), np.abs(ampCisoid)*1./480., np.angle(ampCisoid)
  

def esprit2(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    U, S, V = lg.svd(R)  # 25% of computation time
    #w, v = lg.eig(R)    
    #idx = w.argsort()[::-1]   
    #w = w[idx]
    #v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = U[:-1, :L]
    # Remove first row
    S2 = U[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)

    return f, S[:L]

Using TensorFlow backend.


In [93]:
from time import time
import numpy as np
from pandas import DataFrame

# Bool: Needs to be set by user 
full_anom = 0
anom_samples = 0

x = np.random.randn(4096).astype(np.complex128)
F = 12345.6

f0 = 12345.6
f1 = 1000.6
f2 = 726.6
f3 = 57.3

f0=np.random.uniform(low=0.,high = 10000.)
f1=np.random.uniform(low=0.,high = 10000.)
f2=np.random.uniform(low=0.,high = 10000.)
f3 = np.random.uniform(low=0.,high = 10000.)


c0 = 15.
c1 = 7.
c2 = 10.
c0= np.random.uniform(low=3.,high = 100.)
c1= np.random.uniform(low=3.,high = 100.)
c2= np.random.uniform(low=3.,high = 100.)

fs = 48e3
snr = 20.  # dB
Ntone = 6

t = np.arange(0, 0.01, 1/fs)

nvar = 10**(-snr/10.)



M = [100] * 200  # iterating over block length

py = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortreal = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortcmpl = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fest = []
fest2 = []
cisoidAmp = []
cisoidAmp2 = []
i = 0
for m in M:
    i = i + 1
    # Generate fist tripplet of cisoid  
    # Train data start
    xc = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f2*t)
    xc1 = xc + np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
  
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc1, Ntone//2, M=m, fs=fs, verbose=False)
    fest.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp.append(tmpAmp)
    # Train data end 
    
    # Test section start
    
    # Inject Anomaly
    if not(full_anom or i%20):
      display(i)
      for j in np.arange(anom_samples):
        display(i)
        xc2 = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
        tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
        fest2.append(tmpFest)
        tmpAmp = tmpAmp[0]
        cisoidAmp2.append(tmpAmp) 
    # When not Anomaly use xc as baseline signal and add AWGN - Run through esprit
    xc2 = xc +  np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    if full_anom:
      xc2 = c0*np.exp(1j*2*np.pi*f0*t)+c1*np.exp(1j*2*np.pi*f1*t) +c2*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
    fest2.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp2.append(tmpAmp)  
    # Test section end 

# Create train and test set     
x_train = np.hstack((cisoidAmp,fest))    
x_test = np.hstack((cisoidAmp2,fest2))
    

20

40

60

80

100

120

140

160

180

200

In [94]:
display(cisoidAmp2)
display(cisoidAmp)



[array([85.33349784, 78.82887684,  7.14071248]),
 array([85.33339012, 78.82887815,  7.13955433]),
 array([85.33347453, 78.82884568,  7.14115836]),
 array([85.33351855, 78.82888137,  7.13649624]),
 array([85.33350465, 78.82882527,  7.13808365]),
 array([85.33351675, 78.82896166,  7.13940474]),
 array([85.33350039, 78.82892038,  7.13966011]),
 array([85.3335032 , 78.82884705,  7.14149122]),
 array([85.33350962, 78.82885982,  7.13779136]),
 array([85.33348617, 78.82885419,  7.13915902]),
 array([85.33342962, 78.82890847,  7.14053561]),
 array([85.3334614 , 78.82880077,  7.14042189]),
 array([85.33342316, 78.8288971 ,  7.14127551]),
 array([85.33348178, 78.82885397,  7.13974052]),
 array([85.33347552, 78.82889255,  7.14393675]),
 array([85.33347093, 78.82887862,  7.13601182]),
 array([85.33350249, 78.82887172,  7.14316365]),
 array([85.33350175, 78.82887117,  7.1371909 ]),
 array([85.33355081, 78.82888046,  7.14220212]),
 array([85.33347858, 78.82886009,  7.13591312]),
 array([85.33352978,

[array([85.333543  , 78.82885061,  7.13932366]),
 array([85.33337747, 78.82886989,  7.14359504]),
 array([85.33348675, 78.82886399,  7.13626396]),
 array([85.33351514, 78.8288937 ,  7.13779116]),
 array([85.33358551, 78.82886518,  7.14320719]),
 array([85.33354522, 78.828863  ,  7.13976024]),
 array([85.33350893, 78.82888475,  7.13994807]),
 array([85.33338702, 78.8288351 ,  7.13714926]),
 array([85.33350956, 78.82883059,  7.13734973]),
 array([85.33347529, 78.82890694,  7.13828589]),
 array([85.33345474, 78.82888546,  7.14221298]),
 array([85.33350042, 78.8288311 ,  7.13726568]),
 array([85.33343523, 78.82888258,  7.13568259]),
 array([85.33345831, 78.82883083,  7.13909901]),
 array([85.33352688, 78.82879376,  7.13738529]),
 array([85.33355825, 78.8288587 ,  7.13677425]),
 array([85.333492  , 78.82888631,  7.13775814]),
 array([85.33348713, 78.82888307,  7.13880462]),
 array([85.33341456, 78.82884625,  7.13957253]),
 array([85.33342762, 78.82884079,  7.13856015]),
 array([85.33350696,

In [95]:
display(fest2)
display(fest)



[array([9845.69968893, 6518.75426192, 5539.48052256]),
 array([9845.69230296, 6518.75410907, 5539.49927878]),
 array([9845.69808181, 6518.75789294, 5539.47329938]),
 array([9845.70112186, 6518.75373168, 5539.54876481]),
 array([9845.70015962, 6518.76024988, 5539.52308448]),
 array([9845.70099707, 6518.74416711, 5539.50170081]),
 array([9845.69986511, 6518.74912008, 5539.49756611]),
 array([9845.70005939, 6518.75773422, 5539.4679062 ]),
 array([9845.70050317, 6518.75625121, 5539.52781424]),
 array([9845.69888412, 6518.75690569, 5539.50567891]),
 array([9845.69499928, 6518.75053512, 5539.48338753]),
 array([9845.69717862, 6518.76305714, 5539.48522954]),
 array([9845.69455708, 6518.75187917, 5539.47140133]),
 array([9845.69858127, 6518.75693189, 5539.49626399]),
 array([9845.69814995, 6518.75241686, 5539.42826193]),
 array([9845.6978339 , 6518.75405399, 5539.55659848]),
 array([9845.70001069, 6518.75486133, 5539.44079867]),
 array([9845.69995919, 6518.75492569, 5539.53752885]),
 array([98

[array([9845.70281756, 6518.75732165, 5539.50301356]),
 array([9845.69144295, 6518.75507602, 5539.43380359]),
 array([9845.6989237 , 6518.75576413, 5539.55252133]),
 array([9845.70088568, 6518.7522811 , 5539.52781744]),
 array([9845.70578037, 6518.75562625, 5539.44009269]),
 array([9845.70297198, 6518.75587981, 5539.49594468]),
 array([9845.70045592, 6518.75333391, 5539.4929031 ]),
 array([9845.69209239, 6518.75911709, 5539.53820247]),
 array([9845.70049923, 6518.75963723, 5539.53495941]),
 array([9845.69813442, 6518.75071615, 5539.51981172]),
 array([9845.69672108, 6518.75325087, 5539.45620974]),
 array([9845.69986759, 6518.75957905, 5539.53631913]),
 array([9845.69538331, 6518.75358893, 5539.56192167]),
 array([9845.69696635, 6518.75960941, 5539.50665045]),
 array([9845.70169909, 6518.7638564 , 5539.53438418]),
 array([9845.70387831, 6518.75638105, 5539.54426849]),
 array([9845.69928589, 6518.75315094, 5539.52835173]),
 array([9845.69895024, 6518.75353206, 5539.51141595]),
 array([98

In [42]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6

# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="elu",kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=10e-4))(input_layer)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="elu",kernel_initializer= 'he_normal')(encoder)
decoder = Dense(int(encoding_dim / 2), activation='elu',kernel_initializer= 'he_normal')(encoder)
decoder = Dense(input_dim, activation='elu',kernel_initializer= 'he_normal')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_25 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_26 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 24        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [5]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy_with_masking', factor=0.2,
                              patience=5,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.5229 - acc: 0.6167 - val_loss: 0.6079 - val_acc: 0.7500
Epoch 2/800
180/180 [==============================] - 0s 278us/step - loss: 0.5127 - acc: 0.6278 - val_loss: 0.5990 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 307us/step - loss: 0.5022 - acc: 0.6333 - val_loss: 0.5901 - val_acc: 0.7500
Epoch 4/800
180/180 [==============================] - 0s 290us/step - loss: 0.4927 - acc: 0.6333 - val_loss: 0.5807 - val_acc: 0.7000
Epoch 5/800
 32/180 [====>.........................] - ETA: 0s - loss: 0.4667 - acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:972: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_accuracy_with_masking` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


180/180 [==============================] - 0s 320us/step - loss: 0.4829 - acc: 0.6389 - val_loss: 0.5705 - val_acc: 0.7000
Epoch 6/800
180/180 [==============================] - 0s 299us/step - loss: 0.4734 - acc: 0.6333 - val_loss: 0.5620 - val_acc: 0.6500
Epoch 7/800
180/180 [==============================] - 0s 326us/step - loss: 0.4632 - acc: 0.6556 - val_loss: 0.5519 - val_acc: 0.6500
Epoch 8/800
180/180 [==============================] - 0s 373us/step - loss: 0.4528 - acc: 0.6611 - val_loss: 0.5431 - val_acc: 0.6500
Epoch 9/800
180/180 [==============================] - 0s 320us/step - loss: 0.4432 - acc: 0.6667 - val_loss: 0.5324 - val_acc: 0.6500
Epoch 10/800
180/180 [==============================] - 0s 312us/step - loss: 0.4334 - acc: 0.6833 - val_loss: 0.5234 - val_acc: 0.6500
Epoch 11/800
180/180 [==============================] - 0s 306us/step - loss: 0.4246 - acc: 0.7000 - val_loss: 0.5132 - val_acc: 0.7000
Epoch 12/800
180/180 [==============================] - 0s 313us/

'[test_loss, test_acc]'

[0.11783276438713074, 0.945]

In [96]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

x_train2 = x_train2.astype('float32') / 255.
x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6
elu = keras.layers.ELU(alpha=1.2)
lrlu= keras.layers.LeakyReLU(alpha=0.3)


# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim,kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=75.10e-5))(input_layer)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
decoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
decoder = Dense(input_dim,kernel_initializer= 'he_normal')(decoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
autoencoder2 = Model(inputs=input_layer, outputs=decoder)

autoencoder2.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 6)                 42        
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 6)                 0         
_________________________________________________________________
dense_42 (Dense)             (None, 3)                 21        
_________________________________________________________________
leaky_re_lu_33 (LeakyReLU)   (None, 3)                 0         
_________________________________________________________________
dense_43 (Dense)             (None, 3)                 12        
_________________________________________________________________
leaky_re_lu_34 (LeakyReLU)   (None, 3)                 0         
__________

In [0]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 7s 38ms/step - loss: 0.6340 - acc: 0.5222 - val_loss: 0.6486 - val_acc: 0.4000
Epoch 2/800
180/180 [==============================] - 0s 363us/step - loss: 0.6323 - acc: 0.5167 - val_loss: 0.6465 - val_acc: 0.4000
Epoch 3/800
180/180 [==============================] - 0s 358us/step - loss: 0.6305 - acc: 0.5333 - val_loss: 0.6434 - val_acc: 0.4000
Epoch 4/800
180/180 [==============================] - 0s 359us/step - loss: 0.6292 - acc: 0.5389 - val_loss: 0.6430 - val_acc: 0.4000
Epoch 5/800
180/180 [==============================] - 0s 364us/step - loss: 0.6275 - acc: 0.5611 - val_loss: 0.6400 - val_acc: 0.4000
Epoch 6/800
180/180 [==============================] - 0s 347us/step - loss: 0.6262 - acc: 0.5778 - val_loss: 0.6392 - val_acc: 0.4500
Epoch 7/800
180/180 [==============================] - 0s 354us/step - loss: 0.6250 - acc: 0.6111 - val_loss: 0.6368 - val_acc: 0.5000
Epoch 8/800

'[test_loss, test_acc]'

[0.43619103610515597, 0.945]

In [97]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=200,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 8ms/step - loss: 1.2706 - acc: 0.2556 - val_loss: 0.7939 - val_acc: 0.2000
Epoch 2/800
180/180 [==============================] - 0s 312us/step - loss: 1.2526 - acc: 0.2667 - val_loss: 0.7874 - val_acc: 0.1500
Epoch 3/800
180/180 [==============================] - 0s 299us/step - loss: 1.2362 - acc: 0.2944 - val_loss: 0.7812 - val_acc: 0.1500
Epoch 4/800
180/180 [==============================] - 0s 277us/step - loss: 1.2222 - acc: 0.2889 - val_loss: 0.7698 - val_acc: 0.2000
Epoch 5/800
180/180 [==============================] - 0s 280us/step - loss: 1.2042 - acc: 0.3056 - val_loss: 0.7594 - val_acc: 0.2000
Epoch 6/800
180/180 [==============================] - 0s 311us/step - loss: 1.1890 - acc: 0.3222 - val_loss: 0.7485 - val_acc: 0.2000
Epoch 7/800
180/180 [==============================] - 0s 294us/step - loss: 1.1661 - acc: 0.3333 - val_loss: 0.7373 - val_acc: 0.2000
Epoch 8/800


'[test_loss, test_acc]'

[0.43870301961898805, 0.955]

In [98]:
score2 = autoencoder2.predict( x_test2, batch_size=32, verbose=1,  steps=None)
display('[test_loss, test_acc]')
display(score2)

200/200 [==============================] - 0s 2ms/step


'[test_loss, test_acc]'

array([[ 1.3211937e-01, -6.2396143e-02,  6.5384960e-01,  1.2518707e-01,
        -1.5824040e-03, -4.1008580e-02],
       [-1.0439655e-02, -6.2614374e-02,  8.1916451e-02, -1.0633676e-02,
        -1.9043984e-03, -1.9491628e-02],
       [-4.0084333e-03, -8.5423249e-01,  9.1851139e-01, -4.1424576e-03,
         8.6065662e-01, -4.8470408e-02],
       ...,
       [ 6.2889993e-01, -1.1495882e+00, -2.7839032e-03,  6.2691230e-01,
         1.2589065e+00, -5.7164882e-03],
       [ 2.2926583e+00, -4.7994193e-01,  1.0982301e+00,  2.3071930e+00,
         4.4758952e-01, -5.7572521e-02],
       [ 5.5588728e-01, -7.4451602e-01, -5.8034719e-03,  5.5333906e-01,
         7.3625445e-01,  3.0678082e-01]], dtype=float32)

In [100]:
display(score2[0,:])
display(x_test2[0,:])
display(x_train2[2,:])
display(x_train[3,:])
display(x_test[0,:])

array([ 0.13211937, -0.06239614,  0.6538496 ,  0.12518707, -0.0015824 ,
       -0.04100858], dtype=float32)

array([ 0.1411166 ,  0.09544437,  0.69605088,  0.1378516 , -0.08965288,
       -0.69588502])

array([ 0.03809295, -0.21452784, -1.09779323,  0.03475823,  0.21923955,
        1.09762546])

array([8.53335151e+01, 7.88288937e+01, 7.13779116e+00, 9.84570089e+03,
       6.51875228e+03, 5.53952782e+03])

array([8.53334978e+01, 7.88288768e+01, 7.14071248e+00, 9.84569969e+03,
       6.51875426e+03, 5.53948052e+03])

In [62]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=200,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 6ms/step - loss: 0.3931 - acc: 0.5556 - val_loss: 0.5420 - val_acc: 0.6000
Epoch 2/800
180/180 [==============================] - 0s 262us/step - loss: 0.3901 - acc: 0.5667 - val_loss: 0.5438 - val_acc: 0.5500
Epoch 3/800
180/180 [==============================] - 0s 272us/step - loss: 0.3894 - acc: 0.5556 - val_loss: 0.5399 - val_acc: 0.4500
Epoch 4/800
180/180 [==============================] - 0s 287us/step - loss: 0.3880 - acc: 0.5444 - val_loss: 0.5415 - val_acc: 0.5500
Epoch 5/800
180/180 [==============================] - 0s 254us/step - loss: 0.3873 - acc: 0.5611 - val_loss: 0.5440 - val_acc: 0.5500
Epoch 6/800
180/180 [==============================] - 0s 271us/step - loss: 0.3885 - acc: 0.5667 - val_loss: 0.5405 - val_acc: 0.5500
Epoch 7/800
180/180 [==============================] - 0s 257us/step - loss: 0.3869 - acc: 0.5556 - val_loss: 0.5394 - val_acc: 0.5500
Epoch 8/800


'[test_loss, test_acc]'

[0.16832197070121765, 0.805]

In [46]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 4ms/step - loss: 0.1152 - acc: 0.8667 - val_loss: 0.1742 - val_acc: 0.7000
Epoch 2/800
180/180 [==============================] - 0s 280us/step - loss: 0.1145 - acc: 0.8611 - val_loss: 0.1704 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 265us/step - loss: 0.1134 - acc: 0.8667 - val_loss: 0.1692 - val_acc: 0.8000
Epoch 4/800
180/180 [==============================] - 0s 276us/step - loss: 0.1138 - acc: 0.8611 - val_loss: 0.1701 - val_acc: 0.7000
Epoch 5/800
180/180 [==============================] - 0s 281us/step - loss: 0.1147 - acc: 0.8556 - val_loss: 0.1743 - val_acc: 0.7500
Epoch 6/800
180/180 [==============================] - 0s 267us/step - loss: 0.1144 - acc: 0.8556 - val_loss: 0.1706 - val_acc: 0.7500
Epoch 7/800
180/180 [==============================] - 0s 281us/step - loss: 0.1145 - acc: 0.8556 - val_loss: 0.1695 - val_acc: 0.8000
Epoch 8/800


'[test_loss, test_acc]'

[28.785220642089843, 0.215]

In [69]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=200,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.predict( x_test2, batch_size=32, verbose=1, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 1s 6ms/step - loss: 0.1455 - acc: 0.8778 - val_loss: 0.2047 - val_acc: 0.8500
Epoch 2/800
180/180 [==============================] - 0s 261us/step - loss: 0.1414 - acc: 0.9000 - val_loss: 0.2030 - val_acc: 0.8500
Epoch 3/800
180/180 [==============================] - 0s 260us/step - loss: 0.1412 - acc: 0.9000 - val_loss: 0.2047 - val_acc: 0.9000
Epoch 4/800
180/180 [==============================] - 0s 265us/step - loss: 0.1418 - acc: 0.9111 - val_loss: 0.2071 - val_acc: 0.9000
Epoch 5/800
180/180 [==============================] - 0s 357us/step - loss: 0.1416 - acc: 0.9167 - val_loss: 0.2036 - val_acc: 0.8500
Epoch 6/800
180/180 [==============================] - 0s 265us/step - loss: 0.1415 - acc: 0.9000 - val_loss: 0.2066 - val_acc: 0.9000
Epoch 7/800
180/180 [==============================] - 0s 272us/step - loss: 0.1421 - acc: 0.9000 - val_loss: 0.2031 - val_acc: 0.9000
Epoch 8/800


'[test_loss, test_acc]'

array([[-0.2799706 ,  0.7340043 , -0.57149446,  0.20620513,  0.7250215 ,
         0.545471  ],
       [-0.4654215 , -0.43440944, -0.06398118,  0.5028001 , -0.5480597 ,
        -0.19928044],
       [ 0.7209996 ,  1.810292  ,  0.88380563, -0.6303836 ,  1.99613   ,
        -0.70400417],
       ...,
       [ 0.17465913, -0.18046528, -0.7016069 , -0.2663977 , -0.31282198,
         0.8983585 ],
       [ 1.0703079 , -0.67933416, -0.93009007, -0.6990808 , -0.7755674 ,
         2.3301945 ],
       [ 1.371816  , -0.12273365, -0.60305953, -0.79855275, -0.25935274,
         0.5816963 ]], dtype=float32)

In [75]:
score2 = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score2)

200/200 [==============================] - 0s 129us/step


'[test_loss, test_acc]'

[0.1643379411101341, 0.81]

In [91]:
display(x_train[3,:])
display(x_train[0,:])
display(x_train[5,:])



array([7.98813047e+01, 6.53505302e+01, 1.27098814e+00, 9.01627079e+03,
       4.86375687e+03, 2.22070061e+03])

array([7.98813045e+01, 6.53505343e+01, 1.26891868e+00, 9.01627170e+03,
       4.86376676e+03, 2.22094162e+03])

array([7.98813041e+01, 6.53505316e+01, 1.27635026e+00, 9.01627352e+03,
       4.86375979e+03, 2.22004959e+03])

In [92]:
display(score[0,:])
display(x_test2[0,:])
display(x_train2[2,:])
display(x_train[3,:])
display(x_test[0,:])

array([-0.2799706 ,  0.7340043 , -0.57149446,  0.20620513,  0.7250215 ,
        0.545471  ], dtype=float32)

array([-0.13627917,  0.71676571, -0.56267982,  0.3875516 ,  0.71680732,
        0.574813  ])

array([-0.18443958,  1.23373152, -1.62555058,  0.42156024,  1.3637031 ,
        1.601184  ])

array([7.98813047e+01, 6.53505302e+01, 1.27098814e+00, 9.01627079e+03,
       4.86375687e+03, 2.22070061e+03])

array([7.98813043e+01, 3.41690118e-01, 1.26727822e+00, 9.01627280e+03,
       3.80362979e+02, 2.22112900e+03])

In [0]:
#Results with one sample anomily 
# the duration of the anom can be tuned with anom_samples
full_anom = 0
anom_samples = 1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 9s 48ms/step - loss: 1.5007 - acc: 0.1944 - val_loss: 1.4750 - val_acc: 0.0500
Epoch 2/800
180/180 [==============================] - 0s 353us/step - loss: 1.4188 - acc: 0.1778 - val_loss: 1.3748 - val_acc: 0.1000
Epoch 3/800
180/180 [==============================] - 0s 345us/step - loss: 1.3436 - acc: 0.1778 - val_loss: 1.2763 - val_acc: 0.1000
Epoch 4/800
180/180 [==============================] - 0s 350us/step - loss: 1.2729 - acc: 0.1889 - val_loss: 1.1892 - val_acc: 0.1000
Epoch 5/800
180/180 [==============================] - 0s 372us/step - loss: 1.2061 - acc: 0.2000 - val_loss: 1.1073 - val_acc: 0.0500
Epoch 6/800
180/180 [==============================] - 0s 348us/step - loss: 1.1525 - acc: 0.2222 - val_loss: 1.0312 - val_acc: 0.0500
Epoch 7/800
180/180 [==============================] - 0s 366us/step - loss: 1.0978 - acc: 0.2389 - val_loss: 0.9676 - val_acc: 0.0500
Epoch 8/800

'[test_loss, test_acc]'

[5.602794224875314, 0.276190476332392]